In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from chembl_webresource_client.new_client import new_client
import math

In [2]:
dg_descriptors = pd.read_csv(r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\All_csv_files\drug_descriptors.csv',index_col=0)
dg_descriptors

,molecule_pref_name,molecule_chembl_id,canonical_smiles,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CEFUROXIME,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,12.570377,12.570377,0.140077,-1.353040,0.295008,21.206897,424.391,...,1,0,0,0,0,0,0,0,0,0
1,CEFAZOLIN,CHEMBL1435,Cc1nnc(SCC2=C(C(=O)O)N3C(=O)[C@@H](NC(=O)Cn4cn...,12.598077,12.598077,0.015313,-1.160962,0.405190,21.000000,454.519,...,2,0,0,0,1,0,0,0,0,0
2,MINOCYCLINE,CHEMBL1434,CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(O...,13.493332,13.493332,0.001835,-2.625690,0.398372,29.090909,457.483,...,0,0,0,0,0,0,0,0,0,0
3,DOXYCYCLINE ANHYDROUS,CHEMBL1433,C[C@H]1c2cccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(C...,13.317028,13.317028,0.103720,-2.894779,0.332217,34.375000,444.440,...,0,0,0,0,0,0,0,0,0,0
4,CASPOFUNGIN,CHEMBL1630575,CC[C@H](C)C[C@H](C)CCCCCCCCC(=O)N[C@H]1C[C@@H]...,14.581580,14.581580,0.025749,-2.278500,0.047716,28.636364,1093.331,...,0,0,0,0,0,0,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,GEMIFLOXACIN,CHEMBL430,CO/N=C1\CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)...,14.874521,14.874521,0.033885,-1.336542,0.730424,20.892857,389.387,...,0,0,0,0,0,0,0,0,0,0
90,AZITHROMYCIN,CHEMBL529,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,14.197646,14.197646,0.133072,-1.798815,0.238475,49.615385,748.996,...,0,0,0,0,0,0,0,0,0,0
91,AZTREONAM,CHEMBL158,C[C@H]1[C@H](NC(=O)/C(=N\OC(C)(C)C(=O)O)c2csc(...,12.562142,12.562142,0.034954,-4.756269,0.174788,20.500000,435.440,...,0,0,0,0,0,1,0,0,0,0
92,IMIPENEM,CHEMBL148,C[C@@H](O)[C@H]1C(=O)N2C(C(=O)O)=C(SCCNC=N)C[C...,11.938415,11.938415,0.052549,-1.104254,0.224452,26.100000,299.352,...,1,0,0,0,0,0,0,0,0,0


In [3]:
target_type_filt = pd.read_csv(r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\All_csv_files\activity_set_filt.csv',index_col=0)
target_type_filt

,activity_comment,activity_id,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,canonical_smiles,document_chembl_id,...,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,toid,type,units,uo_units,value
0,NaN,748926,CHEMBL810377,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,1.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,1.00
1,NaN,748927,CHEMBL810376,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,2.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,2.00
2,NaN,748928,CHEMBL883635,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,1.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,1.00
3,NaN,748929,CHEMBL810375,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,2.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,2.00
4,NaN,748930,CHEMBL810374,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,2.00,CHEMBL352,Staphylococcus aureus,Staphylococcus aureus,1280.0,NaN,MIC,ug ml-1,UO_0000274,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179590,NaN,20151113,CHEMBL4513082,Antiviral activity determined as inhibition of...,F,BAO_0000201,BAO_0000218,organism-based format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL4495565,...,0.03,CHEMBL4303835,Severe acute respiratory syndrome coronavirus 2,SARS-CoV-2,2697049.0,NaN,inhibition,%,UO_0000187,0.03
179591,NaN,22071525,CHEMBL4513082,Antiviral activity determined as inhibition of...,F,BAO_0000201,BAO_0000218,organism-based format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL4495565,...,0.03,CHEMBL4303835,Severe acute respiratory syndrome coronavirus 2,SARS-CoV-2,2697049.0,NaN,inhibition,%,UO_0000187,0.03
179592,NaN,25083065,CHEMBL5256281,Antioxidant activity assessed as hydroxyl radi...,F,BAO_0000375,BAO_0000019,assay format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL5252641,...,39.00,CHEMBL2362975,NaN,No relevant target,NaN,NaN,Activity,NaN,NaN,39.00
179593,Summarised AC50 (mean value for measurements w...,25197394,CHEMBL5291879,Compound was evaluated for inhibition of rat S...,F,BAO_0000186,BAO_0000019,assay format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL5291721,...,30000.00,CHEMBL4828,Rattus norvegicus,Synaptic vesicular amine transporter,10116.0,NaN,AC50,uM,UO_0000065,30.00


In [6]:
target_type_filt['standard_units']

0         ug.mL-1
1         ug.mL-1
2         ug.mL-1
3         ug.mL-1
4         ug.mL-1
           ...   
179590          %
179591          %
179592        NaN
179593         nM
179594         nM
Name: standard_units, Length: 179595, dtype: object

In [5]:
result = pd.merge(target_type_filt, dg_descriptors, on='molecule_pref_name', how='left')
display(result)

,activity_comment,activity_id,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,canonical_smiles_x,document_chembl_id,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,NaN,748926,CHEMBL810377,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,1,0,0,0,0,0,0,0,0,0
1,NaN,748927,CHEMBL810376,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,1,0,0,0,0,0,0,0,0,0
2,NaN,748928,CHEMBL883635,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,1,0,0,0,0,0,0,0,0,0
3,NaN,748929,CHEMBL810375,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,1,0,0,0,0,0,0,0,0,0
4,NaN,748930,CHEMBL810374,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179590,NaN,20151113,CHEMBL4513082,Antiviral activity determined as inhibition of...,F,BAO_0000201,BAO_0000218,organism-based format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL4495565,...,0,0,0,0,0,0,0,0,0,0
179591,NaN,22071525,CHEMBL4513082,Antiviral activity determined as inhibition of...,F,BAO_0000201,BAO_0000218,organism-based format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL4495565,...,0,0,0,0,0,0,0,0,0,0
179592,NaN,25083065,CHEMBL5256281,Antioxidant activity assessed as hydroxyl radi...,F,BAO_0000375,BAO_0000019,assay format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL5252641,...,0,0,0,0,0,0,0,0,0,0
179593,Summarised AC50 (mean value for measurements w...,25197394,CHEMBL5291879,Compound was evaluated for inhibition of rat S...,F,BAO_0000186,BAO_0000019,assay format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL5291721,...,0,0,0,0,0,0,0,0,0,0


Объединение сета  дескрипторами для лекарств с таксономией

In [10]:
# # Example dataset
# data = {
#     'standard_unit': ['uM', 'mg/L', 'mg/dL', 'nM', 'mmol/L'],
#     'standard_value': [1000, 100, 10, 500, 1],
#     'mol_wt': [180.18, None, None, 180.18, 180.18]  # Example molecular weights (g/mol)
# }

# df = pd.DataFrame(data)

# Conversion factors to µg/ml
def convert_to_ug_ml(row):
    unit = row['standard_units']
    value = row['standard_value']
    mol_wt = row['MolWt']  # in g/mol

    if unit == 'µg/ml':
        return value
    elif unit == 'mg/ml':
        return value * 1000
    elif unit == 'g/ml':
        return value * 1_000_000
    elif unit == 'mg/L':
        return value
    elif unit == 'mg/dL':
        return value * 10
    elif unit == 'uM' and mol_wt is not None:
        return value * mol_wt  # convert to µg/ml
    elif unit == 'mM' and mol_wt is not None:
        return value * (mol_wt * 1)  # convert to µg/ml
    elif unit == 'mmol/L' and mol_wt is not None:
        return value * (mol_wt)  # convert to µg/ml
    elif unit == 'nM' and mol_wt is not None:
        return value * mol_wt * 0.001  # convert to µg/ml
    elif unit == 'pg/ml':
        return value * 0.001
    elif unit == 'ng/ml':
        return value * 0.001
    elif unit == 'g/dL':
        return value * 10000
    else:
        return None  # Handle cases where conversion is not defined

# Apply conversion
result['value_in_ug_ml'] = result.apply(convert_to_ug_ml, axis=1)

display(result)

,activity_comment,activity_id,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,canonical_smiles_x,document_chembl_id,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,value_in_ug_ml
0,NaN,748926,CHEMBL810377,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,0,0,0,0,0,0,0,0,0,NaN
1,NaN,748927,CHEMBL810376,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,0,0,0,0,0,0,0,0,0,NaN
2,NaN,748928,CHEMBL883635,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,0,0,0,0,0,0,0,0,0,NaN
3,NaN,748929,CHEMBL810375,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,0,0,0,0,0,0,0,0,0,NaN
4,NaN,748930,CHEMBL810374,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179590,NaN,20151113,CHEMBL4513082,Antiviral activity determined as inhibition of...,F,BAO_0000201,BAO_0000218,organism-based format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL4495565,...,0,0,0,0,0,0,0,0,0,NaN
179591,NaN,22071525,CHEMBL4513082,Antiviral activity determined as inhibition of...,F,BAO_0000201,BAO_0000218,organism-based format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL4495565,...,0,0,0,0,0,0,0,0,0,NaN
179592,NaN,25083065,CHEMBL5256281,Antioxidant activity assessed as hydroxyl radi...,F,BAO_0000375,BAO_0000019,assay format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL5252641,...,0,0,0,0,0,0,0,0,0,NaN
179593,Summarised AC50 (mean value for measurements w...,25197394,CHEMBL5291879,Compound was evaluated for inhibition of rat S...,F,BAO_0000186,BAO_0000019,assay format,OC[C@@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,CHEMBL5291721,...,0,0,0,0,0,0,0,0,0,5465.16


In [13]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179595 entries, 0 to 179594
Data columns (total 247 columns):
 #    Column                     Dtype  
---   ------                     -----  
 0    activity_comment           object 
 1    activity_id                int64  
 2    assay_chembl_id            object 
 3    assay_description          object 
 4    assay_type                 object 
 5    bao_endpoint               object 
 6    bao_format                 object 
 7    bao_label                  object 
 8    canonical_smiles_x         object 
 9    document_chembl_id         object 
 10   document_journal           object 
 11   document_year              float64
 12   molecule_chembl_id_x       object 
 13   molecule_pref_name         object 
 14   parent_molecule_chembl_id  object 
 15   potential_duplicate        int64  
 16   qudt_units                 object 
 17   record_id                  int64  
 18   relation                   object 
 19   src_id               

In [16]:
pd.options.display.max_rows = None

In [17]:
# Количество ненулевых значений в каждом столбце
non_null_counts = result.notnull().sum()

# Добавим к информации о каждом столбце количество ненулевых значений
print("\nКоличество ненулевых значений в каждом столбце:")
print(non_null_counts)


Количество ненулевых значений в каждом столбце:
activity_comment              38607
activity_id                  179595
assay_chembl_id              179595
assay_description            179595
assay_type                   179595
bao_endpoint                 179595
bao_format                   179595
bao_label                    179595
canonical_smiles_x           179595
document_chembl_id           179595
document_journal             151590
document_year                151940
molecule_chembl_id_x         179595
molecule_pref_name           179595
parent_molecule_chembl_id    179595
potential_duplicate          179595
qudt_units                   142941
record_id                    179595
relation                     159582
src_id                       179595
standard_flag                179595
standard_relation            159549
standard_type                179595
standard_units               154356
standard_value               159907
target_chembl_id             179595
target_organism

In [19]:
result.to_csv(r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\All_csv_files\initial_dg_descript.csv')
result.to_excel(r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\All_csv_files\initial_dg_descript.xlsx')